In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 20:33 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount
mkdir: `/tmp/wordcount': File exists
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-18 19:12 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 19:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-18 20:33 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-18 19:38 /tmp/wordcount


In [3]:
!hdfs dfs -copyFromLocal *.csv  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/tbl0.csv': File exists
copyFromLocal: `/tmp/wordcount/tbl1.csv': File exists


In [4]:
%%hive

DROP TABLE IF EXISTS tbl0;
DROP TABLE IF EXISTS tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 28.881 seconds
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.369 seconds


In [5]:
%%hive

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.847 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 3.692 seconds


In [6]:
%%hive

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.176 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 2.304 seconds


In [14]:
%%hive
DROP TABLE IF EXISTS lets_upp;
CREATE TABLE lets_upp AS
SELECT b.result
FROM(SELECT a.c1, concat_ws(':', collect_set(UPPER(a.c5))) AS result
FROM(SELECT c1 AS c1, lets AS c5 
FROM tbl0 
LATERAL VIEW explode(c5) tbl0 AS lets) a
GROUP BY a.c1) b;

DROP TABLE IF EXISTS lets_upp;
OK
Time taken: 0.018 seconds
CREATE TABLE lets_upp AS
SELECT b.result
FROM(SELECT a.c1, concat_ws(':', collect_set(UPPER(a.c5))) AS result
FROM(SELECT c1 AS c1, lets AS c5 
FROM tbl0 
LATERAL VIEW explode(c5) tbl0 AS lets) a
GROUP BY a.c1) b;
Query ID = root_20200218205743_590504a2-1553-46d2-ba42-0aebcd5b0199
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582053079980_0019, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0019/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0019
Hadoop job information for Stage-1: number of mappers: 1; number 

In [15]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_upp;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM lets_upp;
Query ID = root_20200218205830_7e01f0ea-55a0-4a88-aa52-982b199fc50d
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582053079980_0020, Tracking URL = http://fbebe19f3edd:8088/proxy/application_1582053079980_0020/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582053079980_0020
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 20:58:46,923 Stage-1 map = 0%,  reduce = 0%
2020-02-18 20:58:56,511 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.4 sec
MapReduce Total cumulative CPU time: 3 seconds 400 msec
Ended Job = job_1582053079980_0020
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-staging_hive

In [16]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [17]:
!hdfs dfs -cat /tmp/output/000000_0 | head

A:E:C:D
A:C
B:D
A:C:E:B:D
D:E:C
A:D
A:E:D
D:A:C:E
B:A:C:E
C:D:E:A:B


In [18]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [19]:
!cat output/000000_0

A:E:C:D
A:C
B:D
A:C:E:B:D
D:E:C
A:D
A:E:D
D:A:C:E
B:A:C:E
C:D:E:A:B
